In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
import math
from typing import List, Tuple, Callable

from app.models.race import Race
from app.models.race_entry import RaceEntry
from app.models.bet import Bet
from app.raceday.bet_strategy.generator import BetGen
import hyperopt
from app.db.session import SessionLocal
import random
from pydantic import BaseModel
from decimal import Decimal
from sqlalchemy.orm import make_transient


In [3]:
db = SessionLocal()

In [4]:
races = db.query(Race).all()
race = races[random.randint(0, len(races)-1)]

print(race)
print(sum([1/entry.latest_odds() for entry in race.active_entries()]))
entries_sorted = race.entries.copy()
entries_sorted.sort(key=lambda x: x.latest_odds())
print(entries_sorted[0])
print(race.win_pool_total, race.place_pool_total, race.show_pool_total)

<Race(id=24549,race_number=5,race_date=datetime.date(2022, 7, 16),track_code='ifh',post_time=datetime.datetime(2022, 7, 16, 20, 59, 29, 240606, tzinfo=datetime.timezone.utc),current_race=True,win_pool_total=37134553.4393764,place_pool_total=47457675.057842776,show_pool_total=36301753.324903816)>
1.0
<RaceEntry(id=401604,name='Welcome',program_no='5',win_pool_total=3865105.4644470476,place_pool_total=5522398.460330129,show_pool_total=5522398.460330129,odds=4.25)>
37134553.4393764 47457675.057842776 36301753.324903816


In [5]:
bet_gen = BetGen(race=race, use_pool_totals=True)

In [6]:
all_bets = bet_gen.all_bets()

In [7]:
all_bets

[<DrZPlaceBet(cost=2, min_reward=0, avg_reward=3.293045235879772, max_reward=11.807783991413887)>,
 <DrZPlaceShowArbBet(cost=8, min_reward=0, avg_reward=11.158950724387388, max_reward=39.306248639442806)>,
 <DrZShowBet(cost=2, min_reward=0, avg_reward=2.72043952114237, max_reward=5.648133579140738)>,
 <DrZPlaceBet(cost=2, min_reward=0, avg_reward=2.7097111054753706, max_reward=8.797385478003887)>,
 <DrZShowBet(cost=2, min_reward=0, avg_reward=2.4357548618898734, max_reward=5.052945590884289)>,
 <WinAllArbBet(cost=12, min_reward=8.5, avg_reward=2.333333333333333, max_reward=26.714285714285715)>]

In [8]:
all_bets2 = bet_gen.all_bets()

In [9]:
all_bets2

[<DrZPlaceBet(cost=2, min_reward=0, avg_reward=3.293045235879772, max_reward=11.807783991413887)>,
 <DrZPlaceShowArbBet(cost=8, min_reward=0, avg_reward=11.158950724387388, max_reward=39.306248639442806)>,
 <DrZShowBet(cost=2, min_reward=0, avg_reward=2.72043952114237, max_reward=5.648133579140738)>,
 <DrZPlaceBet(cost=2, min_reward=0, avg_reward=2.7097111054753706, max_reward=8.797385478003887)>,
 <DrZShowBet(cost=2, min_reward=0, avg_reward=2.4357548618898734, max_reward=5.052945590884289)>,
 <WinAllArbBet(cost=12, min_reward=8.5, avg_reward=2.333333333333333, max_reward=26.714285714285715)>]

In [24]:
bet1 = all_bets[1].result().to_bet_db()

In [17]:
bet2 = all_bets2[1].result().to_bet_db()

In [25]:
bet1.md5_hash().hexdigest()

'34d2e3c08b3a7136243f4a613fb40a2d'

In [22]:
bet2.md5_hash().hexdigest()

'34d2e3c08b3a7136243f4a613fb40a2d'

In [30]:
bet1.sub_bets[0].avg_reward

2.7097111054753706

In [19]:
bet

'd2e443b09f94ecd6502c13a30c8860bb'